In [1]:
import pandas as pd
import time
import os
import matplotlib.pyplot as plt
import descartes
import geopandas as gpd
from shapely.geometry import Point, Polygon

# To display inline image below
import IPython.display

import requests

# For storing png files in memory
import io

# For generating GIF
import imageio

%matplotlib inline

<br />
<br />
<br />
<br />
<br />

## Globals

In [2]:
# Top value to use in scale, 0 = mean + 2 std devs
max_value = 0

# Bottom value to use in scale, should be zero
min_value = 0

# SANDAG API Link
sd_api = "https://opendata.arcgis.com/datasets/854d7e48e3dc451aa93b9daf82789089_0.geojson"

# Zipcode shape file link
zip_shape_full = "https://github.com/mulroony/State-zip-code-GeoJSON/raw/master/ca_california_zip_codes_geo.min.json"

# File to write gif to. Leave blank to just render inline
gif_path = "/tmp/SD_Covid_cases.gif"
#gif_path = ""

# Select ColorMap: https://matplotlib.org/3.2.1/tutorials/colors/colormaps.html
color_map = 'YlOrRd'

<br />
<br />

<br />

<br />

<br />


## Get info from SANDAG API

In [3]:
r = requests.get(sd_api)
rd =  [ _r['properties'] for _r in r.json()['features'] ]

case_df = pd.DataFrame(rd)

# Cleanup, reduce mem
del [r, rd]

case_df.head()


,FID,zipcode_zip,ziptext,case_count,updatedate,created_date,created_user,last_edited_date,last_edited_user,globalid
0,1,91901,91901,1.0,2020/04/01 08:00:00,2020/04/11 18:43:45,Ross.Martin@sdcounty.ca.gov,2020/04/11 18:43:45,Ross.Martin@sdcounty.ca.gov,{87C2AEFB-2AB8-473D-BE60-B021F0AD990A}
1,2,91902,91902,9.0,2020/04/01 08:00:00,2020/04/11 18:43:45,Ross.Martin@sdcounty.ca.gov,2020/04/11 18:43:45,Ross.Martin@sdcounty.ca.gov,{0BBF31F5-A98F-4A42-869E-F425379D6BA8}
2,3,91910,91910,23.0,2020/04/01 08:00:00,2020/04/11 18:43:45,Ross.Martin@sdcounty.ca.gov,2020/04/11 18:43:45,Ross.Martin@sdcounty.ca.gov,{94FEB6AB-AB77-4534-9692-CF8DFC823461}
3,4,91911,91911,21.0,2020/04/01 08:00:00,2020/04/11 18:43:45,Ross.Martin@sdcounty.ca.gov,2020/04/11 18:43:45,Ross.Martin@sdcounty.ca.gov,{83793CC0-427A-4747-9BDD-5D79A947C3FF}
4,5,91913,91913,20.0,2020/04/01 08:00:00,2020/04/11 18:43:45,Ross.Martin@sdcounty.ca.gov,2020/04/11 18:43:45,Ross.Martin@sdcounty.ca.gov,{597D4C08-7AD1-49D2-811D-D54EE7C2BB13}


<br/>
<br/>
<br/>

 
 
## Get zipcodes in COVID-19 data so we can reduce our shape file

In [4]:
known_zips = list(case_df['ziptext'].unique())
print("Zipcodes in data: %s"%(len(known_zips)))

Zipcodes in data: 113


<br/>
<br/>
<br/>
<br/>



## Get zip shape file remove non-SD zips, this file is big...

Got API link from: https://sdgis-sandag.opendata.arcgis.com/datasets/covid-19-statistics-by-zip-code

In [5]:
r = requests.get(zip_shape_full)
rd = r.json()

zip_shape_known = {}
zip_shape_known['type'] = rd['type']
zip_shape_known['features'] = []

for i in rd['features']:
    if i['properties']['ZCTA5CE10'] in known_zips:
        zip_shape_known['features'].append(i)
        
print("Found %s matching zip codes in shape file"%(len(zip_shape_known['features'])))
  
del [r, rd]

Found 108 matching zip codes in shape file


<br />
<br />
<br />
<br />
<br />
## Create gdf to hold zipcode shape data in dataframe

In [6]:
gdf = gpd.GeoDataFrame.from_features(zip_shape_known)
gdf.rename(columns={'ZCTA5CE10': 'zipcode'}, inplace=True)
gdf.set_index('zipcode',inplace=True)
gdf.head()

,geometry,STATEFP10,GEOID10,CLASSFP10,MTFCC10,FUNCSTAT10,ALAND10,AWATER10,INTPTLAT10,INTPTLON10,PARTFLG10
zipcode,,,,,,,,,,,
92014,"MULTIPOLYGON (((-117.25330 32.97994, -117.2532...",06,0692014,B5,G6350,S,15615216,1228040,+32.9656171,-117.2482956,N
92020,"POLYGON ((-116.95575 32.76295, -116.95577 32.7...",06,0692020,B5,G6350,S,28841078,0,+32.7955100,-116.9697215,N
92111,"POLYGON ((-117.16476 32.82585, -117.16513 32.8...",06,0692111,B5,G6350,S,21910604,0,+32.8064786,-117.1688636,N
92108,"MULTIPOLYGON (((-117.10392 32.77212, -117.1042...",06,0692108,B5,G6350,S,11094650,307194,+32.7740459,-117.1424539,N
92037,"POLYGON ((-117.28240 32.82618, -117.28255 32.8...",06,0692037,B5,G6350,S,33871798,5226853,+32.8563468,-117.2500579,N


<br />
<br />
<br />
<br />
<br />

## Prep case_df. Remove columns, rename, format data, etc

In [7]:
# Drop time from date, not useful
case_df['date'] = case_df['updatedate'].apply(lambda x: x.split(" ")[0])

# Drop unused fields
case_df.drop(inplace=True, columns=['FID',
                                    'zipcode_zip',
                                    'created_date',
                                    'updatedate',
                                    'created_date', 
                                    'created_user', 
                                    'last_edited_date', 
                                    'last_edited_user',
                                    'globalid'])

# Missing data becomes zeros
case_df.fillna(0, inplace=True)

# Rename column
case_df.rename(columns={'ziptext': 'zipcode'}, inplace=True)

# Drop duplicates, have seen some
case_df.drop_duplicates(subset=['zipcode','date'], inplace=True)

<br />
<br />
<br />
<br />
<br />

## Pivot case_df to be easier to use. rows = zips, cols = dates

In [8]:
# Ugly, but creates table I want
case_df = case_df.groupby(['zipcode', 'date']).sum().unstack().fillna(0)

# End up with nested column name, remove it
case_df.columns = case_df.columns.droplevel()

case_df.head()

date,2020/03/30,2020/03/31,2020/04/01,2020/04/02,2020/04/03,2020/04/04,2020/04/05,2020/04/06,2020/04/07,2020/04/08,...,2020/05/17,2020/05/18,2020/05/19,2020/05/20,2020/05/21,2020/05/22,2020/05/23,2020/05/24,2020/05/25,2020/05/26
zipcode,,,,,,,,,,,,,,,,,,,,,
91901,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,8.0,8.0,9.0,9.0,9.0,10.0,10.0,10.0,10.0,10.0
91902,8.0,8.0,9.0,10.0,10.0,11.0,11.0,14.0,16.0,16.0,...,36.0,38.0,37.0,37.0,37.0,38.0,38.0,38.0,38.0,40.0
91905,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
91906,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
91910,17.0,21.0,23.0,28.0,28.0,30.0,30.0,34.0,39.0,43.0,...,272.0,278.0,286.0,293.0,296.0,305.0,313.0,314.0,319.0,327.0


<br />
<br />
<br />
<br />
<br />

## Find difference between days, modify case_df

In [9]:

# Create super list of all case values so we can get stats if we are going to use it
if max_value == 0:
    _tmp_case_list = []

# Not necessary, but can't hurt
dates = sorted(case_df.columns.values)
    
# subtract tomorrow from today, and set that as the value for today. repeat, skipping last day...
for i in range(len(dates)-1):
    today = dates[i]
    tomorrow = dates[i+1]
    
    case_df[today] = case_df[tomorrow] - case_df[today]
    
#     #Uncomment to find all negative values. Happens due to adjusting the numbers, we handle it
#     #Good to do though
#     _tmp_df = case_df[today].apply(lambda x: x if x < -1 else None).dropna()
#     if _tmp_df.values.size > 0:
#         print("%s"%(today))
#         print(_tmp_df)

    if max_value == 0:
        _tmp_case_list += list(case_df[today].values)

if max_value == 0:
    _tmp_case_df = pd.DataFrame(_tmp_case_list)
    max_value = int(_tmp_case_df.mean()[0] + (2 * _tmp_case_df.std()[0]))
    
print("max_value = %s"%(max_value))

# Limit values based on max / min
for i in dates[:-1]:
    case_df[i] = case_df[i].apply(lambda x: min_value if x < min_value else x)
    case_df[i] = case_df[i].apply(lambda x: max_value if x > max_value else x)

# Remove last day
case_df.drop(inplace=True, columns=[case_df.columns[-1]])

case_df.head()

max_value = 5


date,2020/03/30,2020/03/31,2020/04/01,2020/04/02,2020/04/03,2020/04/04,2020/04/05,2020/04/06,2020/04/07,2020/04/08,...,2020/05/16,2020/05/17,2020/05/18,2020/05/19,2020/05/20,2020/05/21,2020/05/22,2020/05/23,2020/05/24,2020/05/25
zipcode,,,,,,,,,,,,,,,,,,,,,
91901,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
91902,0.0,1.0,1.0,0.0,1.0,0.0,3.0,2.0,0.0,0.0,...,0.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0
91905,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
91906,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
91910,4.0,2.0,5.0,0.0,2.0,0.0,4.0,5.0,4.0,1.0,...,5.0,5.0,5.0,5.0,3.0,5.0,5.0,1.0,5.0,5.0


<br />
<br />
<br />
<br />
<br />

## Merge shape file with zipcodes gdf and case_df, create case_gdf

In [10]:
case_gdf = gdf.merge(case_df, left_on='zipcode', right_on='zipcode')
case_gdf.head()

,geometry,STATEFP10,GEOID10,CLASSFP10,MTFCC10,FUNCSTAT10,ALAND10,AWATER10,INTPTLAT10,INTPTLON10,...,2020/05/16,2020/05/17,2020/05/18,2020/05/19,2020/05/20,2020/05/21,2020/05/22,2020/05/23,2020/05/24,2020/05/25
zipcode,,,,,,,,,,,,,,,,,,,,,
92014,"MULTIPOLYGON (((-117.25330 32.97994, -117.2532...",06,0692014,B5,G6350,S,15615216,1228040,+32.9656171,-117.2482956,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
92020,"POLYGON ((-116.95575 32.76295, -116.95577 32.7...",06,0692020,B5,G6350,S,28841078,0,+32.7955100,-116.9697215,...,5.0,4.0,1.0,5.0,5.0,0.0,5.0,5.0,5.0,2.0
92111,"POLYGON ((-117.16476 32.82585, -117.16513 32.8...",06,0692111,B5,G6350,S,21910604,0,+32.8064786,-117.1688636,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,2.0
92108,"MULTIPOLYGON (((-117.10392 32.77212, -117.1042...",06,0692108,B5,G6350,S,11094650,307194,+32.7740459,-117.1424539,...,1.0,1.0,2.0,0.0,1.0,0.0,2.0,0.0,0.0,1.0
92037,"POLYGON ((-117.28240 32.82618, -117.28255 32.8...",06,0692037,B5,G6350,S,33871798,5226853,+32.8563468,-117.2500579,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


<br />
<br />
<br />
<br />
<br />

## Generate in memory images PNGs

In [11]:
%%time
output_files = []

for idx in dates[:-1]:

    # Create inmemory file
    output_files.append(io.BytesIO())
    
    fig = case_gdf.plot(cmap=color_map, 
             column=idx,
             linewidth=0.8, 
             edgecolor='0.8',
             vmax=int(max_value),
             vmin=min_value,
             legend=True,
             figsize=(14,8))

    fig.axis('off')

    fig.annotate("Daily Cases COVID-19 : %s - %s"%(dates[0],dates[-2]),
                xy=(.1, .9), xycoords='figure fraction',
                horizontalalignment='left', verticalalignment='top',
                fontsize=20)
    fig.annotate("""P. Mulrooney <mulroony@gmail.com>
* Upper case count limited to mean + 2 std devs
* Missing replaced with zeros
* Decreases between days set to zero
* https://sdgis-sandag.opendata.arcgis.com/datasets/covid-19-statistics-by-zip-code""",
                xy=(.5, .1), xycoords='figure fraction',
                horizontalalignment='left', verticalalignment='top',
                fontsize=8)
    
    fig.annotate(idx,
                xy=(0.1, .1), xycoords='figure fraction',
                horizontalalignment='left', verticalalignment='top',
                fontsize=20)
    
    fig.annotate(idx,
                xy=(0.1, .1), xycoords='figure fraction',
                horizontalalignment='left', verticalalignment='top',
                fontsize=20)


    chart = fig.get_figure()
    chart.savefig(output_files[-1], dpi=150)
    plt.close('all')
    
    output_files[-1].seek(0)

print("Generated %s in memory PNG files\n"%(len(output_files)))

Generated 57 in memory PNG files

CPU times: user 21.3 s, sys: 568 ms, total: 21.9 s
Wall time: 22 s


<br />
<br />
<br />
<br />
<br />

## Generate in GIF

In [12]:
images = []
for output_file in output_files:
    images.append(imageio.imread(output_file))

if not gif_path:
    gif_path = io.BytesIO()

imageio.mimsave(gif_path, images, format="gif", duration=0.75, loop=1)

if gif_path.__class__ != str:
    gif_path.seek(0)
    IPython.display.display(IPython.display.Image(data=gif_path.getvalue()))
else:
    IPython.display.Image(gif_path)